In [15]:
import pandas as pd

Import all the files

In [16]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [17]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [18]:
#Data merges on user_id columns
visit_cart_left_merge = pd.merge(visits, cart, how='left')

Step 3: How long is `visits_cart`?

In [98]:
total_visits = len(visit_cart_left_merge)
#print(total_visits)

Step 4: How many timestamps are null for `cart_time`?

In [97]:
#number of null time stamps
null_cart_time = len(visit_cart_left_merge[visit_cart_left_merge.cart_time.isnull()])
print(null_cart_time)

1652


Step 5: What percentage only visited?

In [99]:
#what percent of people only visited
num_only_visited = null_cart_time / total_visits * 100
print(num_only_visited)

82.6


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [100]:
#left merge cart and checkout
cart_to_checkout = pd.merge(cart,checkout, how='left')
print(cart_to_checkout.count())

#how many people didn't check out with items in their carts
didnt_check_out = len(cart_to_checkout[cart_to_checkout.checkout_time.isnull()])

#how many people have an active cart
active_cart = len(cart_to_checkout.cart_time)

#find the percentage of people that did not check out with active carts
percent_not_checkout = (didnt_check_out/active_cart) * 100
percent_not_checkout = round(percent_not_checkout, 2)
print(percent_not_checkout,'%')

user_id          348
cart_time        348
checkout_time    226
dtype: int64
35.06 %


Step 7: Merge it all together

In [95]:
all_data = visit_cart_left_merge\
    .merge(cart_to_checkout, how='left')\
    .merge(purchase, how='left')
print(all_data.count())

user_id          2108
visit_time       2108
cart_time         456
checkout_time     334
purchase_time     252
dtype: int64


Step 8: % of users who got to checkout but did not purchase

In [65]:
#number of people who checked out
num_checked_out = all_data.checkout_time.count()
#334

#number who purchased
num_purchased = all_data.purchase_time.count()
#252

num_not_purchase = num_checked_out - num_purchased
#82

#I've relized that I could've done this instead to reduce the amount of lines.
##  num_not_purchase = len(all_data[all_data.purchase_time.isnull()])


perc_not_purchase = num_not_purchase/num_checked_out * 100
perc_not_purchase = round(perc_not_purchase, 2)
print(perc_not_purchase)


24.55


Step 9: check each part of the funnel, let's print all 3 of them again

In [102]:
print("{} percent of users who visited the page did not add anything to their cart".format(num_only_visited))
print("{} percent of users added to their cart but didn't check out".format(percent_not_checkout))
print("{} percent of users who made it to checkout but didn't purchase anything".format(perc_not_purchase))

82.6 percent of users who visited the page did not add anything to their cart
35.06 percent of users added to their cart but didn't check out
24.55 percent of users who made it to checkout but didn't purchase anything


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [104]:
all_data['time_to_purchase'] = \
    all_data.purchase_time - all_data.visit_time

Step 11: examine the results

In [107]:
print(all_data[~all_data.time_to_purchase.isnull()].head(5))

                                 user_id          visit_time  \
1   0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
14  486480e2-98c3-4d51-8f4b-b1c07228ce84 2017-01-27 16:34:00   
48  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
49  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
65  ab0125fc-9493-4f59-ad70-24ad264a3a0c 2017-11-18 03:21:00   

             cart_time       checkout_time       purchase_time  \
1  2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00   
14 2017-01-27 16:44:00 2017-01-27 17:10:00 2017-01-27 17:12:00   
48 2017-01-08 20:38:00 2017-01-08 20:52:00 2017-01-08 21:02:00   
49 2017-01-08 20:38:00 2017-01-08 20:52:00 2017-01-08 21:21:00   
65 2017-11-18 03:33:00 2017-11-18 03:57:00 2017-11-18 04:21:00   

   time_to_purchase  
1   0 days 00:44:00  
14  0 days 00:38:00  
48  0 days 00:41:00  
49  0 days 01:00:00  
65  0 days 01:00:00  


Step 12: average time to purchase

In [108]:
avg_purchase_time = all_data.time_to_purchase.mean()

In [109]:
print(avg_purchase_time)

0 days 00:43:12.380952380
